# Modelo 3 - Tiempos Contratados

![Diagrama 3](Diagrama3.png)

### Leer los datos de excel

In [1]:
import pandas as pd

#Funcion para leer los datos de los pacientes
def leer_datos():
    # Leer el archivo de excel
    df = pd.read_excel('DT3.xlsx', sheet_name='Pacientes')
    df1 = pd.read_excel("DT3.xlsx", sheet_name="Medicos")
    
    # Crear un diccionario para almacenar los datos de los pacientes
    pacientesDict = {}
    medicos = {}

    # Iterar sobre las filas del DataFrame
    for index, row in df.iterrows():
        llave = row["ID Servicio"]
        valor1 = row["Llegada"]
        valor2 = row["Atencion"]
        valor3 = row["Zona"]
        valor4 = row["Entidad"]

        # Agregar valores a la lista correspondiente en el diccionario
        if llave in pacientesDict:
            pacientesDict[llave].append([valor1, valor2, valor3, valor4])
        else:
            pacientesDict[llave] = [int(valor1), int(valor2), int(valor3), str(valor4)]
    pacientes = pacientesDict

    # Iterar sobre las filas del DataFrame
    for index, row in df1.iterrows():
        llave = row["ID Medico"]
        valor1 = 0
        valor2 = 0
        valor3 = row["Zona"]
    
        # Agregar valores a la lista correspondiente en el diccionario
        if llave in medicos:
            medicos[llave].append([valor1, valor2, valor3])
        else:
            medicos[llave] = [int(valor1), int(valor2), int(valor3)]
    doctores = medicos
    
    
    return doctores, pacientes

In [2]:
doctores = leer_datos()[0]
pacientes = leer_datos()[1]

### Doctores en la zona

In [3]:
global doctores
def doctores_en_zona(zona):
    doctores_en_zona = []
    for j in doctores:
        if doctores[j][2] == zona:
            doctores_en_zona.append(j)
    return doctores_en_zona

### Cambiar datos para Gantt

In [4]:
import datetime

def convertMin(minutos):
    # Obtenemos la fecha de hoy
    fecha_actual = datetime.date.today()
    # Combinamos la fecha de hoy con la hora 0:00:00
    fecha_hora = datetime.datetime.combine(fecha_actual, datetime.time.min) + datetime.timedelta(minutes=minutos)
    # Convertimos la fecha y hora a formato de cadena
    fecha_hora_str = fecha_hora.strftime("%Y-%m-%d %H:%M:%S")
    return fecha_hora_str

### Asignación de los médicos

In [5]:
global doctores

import pandas as pd

# Función para asignar doctor a paciente
def asignar_doctor(atencion, zona, minuto, estadoMedicos, gantt, IDservicio, TipoCliente):

    # diccionario con los tipos de cliente y el tiempo de atencion esperado
    Clientes = { 
        "EPS SANITAS": 60,
        "MEDISANITAS": 120,
        "FAMISANAR": 180,
        "PARTICULARES": 60,
        "COLSANITAS": 120,
        "IKE ASISTENCIA": 180,
        "COOMEVA": 60,
        "ASISMED": 120,
        "SALUD TOTAL EPS": 180,
        "UNISALUD": 60,
        "SURA": 120,
        "MEDPLUS MP": 180
    }

    #Actualizar el estado de los doctores
    if minuto > 0:
        for j in doctores:
            if minuto >= doctores[j][1]:
                doctores[j][0] = 0
                doctores[j][1] = 0
    doctores_zona = doctores_en_zona(zona)
    
    # Lista de posible asignacion
    posibleAsignacion = []
    # Variable para salir del ciclo
    salir = False

    # 1. ¿Hay doctores dentro de la zona del paciente?
    if len(doctores_zona) > 0:
        #1. SI
        for j in doctores_zona:
            #2. ¿Hay doctores disponibles en la zona?
            if doctores[j][0] == 0:
                #2. SI
                #3. Se elige el primer doctor disponible en la zona
                # Se analiza el tiempo estimado para atender al cliente teniendo en cuenta que si esta dentro de la zona son 20 min de traslado
                tiempoEstimado = doctores[j][1] + 20
                # Se agrega el doctor a una lista de posible asignacion
                posibleAsignacion.append((j, tiempoEstimado))
                #4. ¿Cumple con el tiempo de servicio del cliente?
                if tiempoEstimado <= Clientes[TipoCliente]:
                    #4. SI
                    #5. Asignar doctor al paciente
                    doctores[j][0] = 1
                    # Asignar el ultimo minuto en el que esta ocupado el doctor
                    doctores[j][1] = minuto + atencion
                    gantt.append({'Solicitud': IDservicio, 'Médico': j, 'Inicio': convertMin(minuto), 'Fin': convertMin(minuto + atencion)})
                    salir = True
                    break
                else:
                    #4. NO
                    #6. Se busca primer doctor disponible fuera de la zona
                    doc_list = sorted(doctores, key=lambda k: doctores[k][1])
                    for j in doc_list:
                        # Revisar si el doctor esta disponible en orden de lista
                        if doctores[j][0] == 0:
                            # Se analiza el tiempo estimado para atender al cliente teniendo en cuenta que si esta fuera de la zona son 40 min de traslado
                            tiempoEstimado = doctores[j][1] + 40
                            # Agregar el doctor a la lista de posible asignacion
                            posibleAsignacion.append((j, tiempoEstimado))

                            #7. ¿Cumple con el tiempo de servicio del cliente?
                            if tiempoEstimado <= Clientes[TipoCliente]:
                                #8. Reasignar zona del doctor
                                doctores[j][2] = zona
                                #9. Asignar doctor al paciente
                                doctores[j][0] = 1

                                # Asignar el ultimo minuto en el que esta ocupado el doctor
                                doctores[j][1] = minuto + atencion
                                # Agregar al gantt
                                gantt.append({'Solicitud': IDservicio, 'Médico': j, 'Inicio': convertMin(minuto), 'Fin': convertMin(minuto + atencion)})
                                salir = True
                                break
                            else:
                                #7. NO
                                #10. Se asigna el doctor que menos se demore de la lista posibleAsignacion
                                # definir el doctor con menor tiempo de atencion
                                menorTiempo = float('inf')
                                doctorMenorTiempo = posibleAsignacion[0]
                                for i in posibleAsignacion:
                                    if i[1] < menorTiempo:
                                        menorTiempo = i[1]
                                        doctorMenorTiempo = i[0]
                                # Asignar doctor al paciente
                                doctores[doctorMenorTiempo][0] = 1
                                # Asignar el ultimo minuto en el que esta ocupado el doctor
                                doctores[doctorMenorTiempo][1] = minuto + atencion
                                # Reasignar zona del doctor
                                doctores[doctorMenorTiempo][2] = zona
                                # Agregar al gantt
                                gantt.append({'Solicitud': IDservicio, 'Médico': doctorMenorTiempo, 'Inicio': convertMin(minuto), 'Fin': convertMin(minuto + atencion)})
                                salir = True
                                break
                    if salir:
                        break
            else:
                #2. NO
                #11. Se busca primer doctor disponible fuera de la zona
                doc_list = sorted(doctores, key=lambda k: doctores[k][1])
                for j in doc_list:
                    # Revisar si el doctor esta disponible en orden de lista
                    if doctores[j][0] == 0:
                        # Se analiza el tiempo estimado para atender al cliente teniendo en cuenta que si esta fuera de la zona son 40 min de traslado
                        tiempoEstimado = doctores[j][1] + 40
                        # Agregar el doctor a la lista de posible asignacion
                        posibleAsignacion.append((j, tiempoEstimado))
                        break
                #12. Comparar el tiempo de traslado del doctor desocupado con el tiempo estimado de finalización de las consultas de los doctores dentro de la zona más su tiempo de traslado
                for i in doctores_zona:
                    if doctores[i][1] + 20 < tiempoEstimado:
                        tiempoEstimado = doctores[i][1] + 20
                        posibleAsignacion.append((i, tiempoEstimado))
                #Escoger el doctor con menor tiempo de traslado
                menorTiempo = float('inf')
                doctorMenorTiempo = posibleAsignacion[0]
                for i in posibleAsignacion:
                    if i[1] < menorTiempo:
                        menorTiempo = i[1]
                        doctorMenorTiempo = i[0]

                # 13. Reasignar zona del doctor
                doctores[doctorMenorTiempo][2] = zona
                # 14. Asignar doctor al paciente
                doctores[doctorMenorTiempo][0] = 1

                # Asignar el ultimo minuto en el que esta ocupado el doctor
                doctores[doctorMenorTiempo][1] = minuto + atencion
                # Agregar al gantt
                gantt.append({'Solicitud': IDservicio, 'Médico': doctorMenorTiempo, 'Inicio': convertMin(minuto), 'Fin': convertMin(minuto + atencion)})
                break
    else:
        #1. NO
        #15. Se busca primer doctor disponible fuera de la zona
        doc_list = sorted(doctores, key=lambda k: doctores[k][1])
        for j in doc_list:
            # Revisar si el doctor esta disponible en orden de lista
            if doctores[j][0] == 0:
                # 16. Reasignar zona del doctor
                doctores[j][2] = zona
                # 17. Asignar doctor al paciente
                doctores[j][0] = 1

                # Asignar el ultimo minuto en el que esta ocupado el doctor
                doctores[j][1] = minuto + atencion
                # Agregar al gantt
                gantt.append({'Solicitud': IDservicio, 'Médico': j, 'Inicio': convertMin(minuto), 'Fin': convertMin(minuto + atencion)})
                break

    # Actualizar el estado de los doctores en el DataFrame
    for j in doctores:
        if minuto == doctores[j][1]:
            doctores[j][0] = 0

    # Actualizar el estado de los médicos en el DataFrame
    estado = {'Minuto': minuto}

    for j in doctores:
        estado[f'Doctor {j}'] = 'Ocupado' if doctores[j][0] == 1 else 'Disponible'

    # Agregar el estado de los médicos al DataFrame
    estadoMedicos = estadoMedicos = pd.concat([estadoMedicos, pd.DataFrame(estado, index=[0])], ignore_index=True)

    return estadoMedicos, gantt

### Simular Pacientes

In [6]:
import pandas as pd
import openpyxl

global doctores

# Función principal para asignar médicos a pacientes
def asignacion_doctor1():
    pacientes = leer_datos()[1]

    # Crear DataFrame para el estado de los médicos
    estadoMedicos = pd.DataFrame(columns=['Minuto'] + [f'Doctor {i}' for i in doctores])
    gantt = []

    for i in pacientes:
        # Asignar doctor a paciente
        atencion = pacientes[i][1]
        minuto = pacientes[i][0]
        zona = pacientes[i][2]
        IDservicio = i
        TipoCliente = pacientes[i][3]
        estadoMedicos, gantt = asignar_doctor(atencion, zona, minuto, estadoMedicos, gantt, IDservicio, TipoCliente)

    return gantt

# Prueba para 02/06/2022

In [7]:
gantt = asignacion_doctor1()

### Gantt general

In [8]:
import plotly.express as px
import pandas as pd
#Poner los datos de la lista en un dataframe
df1 = pd.DataFrame(gantt)
# poner los datos en el formato que requiere plotly poner la columna solicitud como texto
df1['Solicitud'] = df1['Solicitud'].astype(str)
# poner la columna médico como texto
df1['Médico'] = df1['Médico'].astype(str)

fig = px.timeline(df1, x_start="Inicio", x_end="Fin", y="Médico", color="Médico", 
                  title="Solicitudes atendidas por médicos")
fig.update_layout(xaxis_title="Tiempo", yaxis_title="Médico")
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(rangeslider_visible=True)
# Hacer el grafico más grande para que se vea mejor
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
)
fig.show()


In [9]:
# Encontrar la cantidad de solicitudes atendidas por cada médico
def solicitudes_atendidas(gantt):
    solicitudes = {}

    for i in gantt:
        if i['Médico'] not in solicitudes:
            solicitudes[i['Médico']] = 1
        else:
            solicitudes[i['Médico']] += 1

    return solicitudes

solicitudes = solicitudes_atendidas(gantt)
print(solicitudes)

# Calcular la desviación estándar de las solicitudes atendidas por cada médico
def desviacion_estandar(solicitudes):
    # Calcular la media de las solicitudes atendidas por cada médico
    media = sum(solicitudes.values())/len(solicitudes)

    # Calcular la varianza de las solicitudes atendidas por cada médico
    varianza = sum([(i - media)**2 for i in solicitudes.values()])/len(solicitudes)

    # Calcular la desviación estándar de las solicitudes atendidas por cada médico
    desviacion = varianza**(1/2)

    return desviacion

desviacion = desviacion_estandar(solicitudes)
print(desviacion)

# calcular solicited promedio por médico
def solicitudes_promedio(solicitudes):
    promedio = sum(solicitudes.values())/len(solicitudes)
    return promedio

promedio = solicitudes_promedio(solicitudes)
print(promedio)

# Calcular total de solicitudes atendidas
def total_solicitudes(solicitudes):
    total = sum(solicitudes.values())
    return total

total = total_solicitudes(solicitudes)
print(total)

{1018472151: 29, 1015437933: 22, 80073352: 21, 52997773: 13, 79955886: 8, 1015414697: 18, 1019088914: 12, 1020777651: 9, 1018446151: 13, 80185764: 12, 39779707: 5, 1127250183: 11, 52200795: 4, 1024468225: 6, 1121853934: 12, 1016039086: 5, 80075437: 13, 80383487: 5, 1020803066: 5, 57293715: 9, 1053327980: 8, 1020808271: 7, 80727764: 1, 1098635342: 7, 1015405667: 3, 1083012532: 3, 1082996581: 1, 1117504115: 8, 1083026203: 6, 1085310672: 10, 1095825225: 1, 1018440480: 4, 1032491705: 1, 1014217039: 2, 1014266018: 3, 1098697055: 3, 85488148: 3, 1140888504: 2, 1019074166: 1}
6.330789481850651
7.846153846153846
306
